# Kafka Message Consuming

In [5]:
from confluent_kafka import Consumer, KafkaError, KafkaException
import json
import time

In [7]:
def read_config():
  # reads the client configuration from client.properties
  # and returns it as a key-value map
  config = {}
  with open("client.properties") as fh:
    for line in fh:
      line = line.strip()
      if len(line) != 0 and line[0] != "#":
        parameter, value = line.strip().split('=', 1)
        config[parameter] = value.strip()
  return config

config = read_config()
config["group.id"] = "customer_group"
config["auto.offset.reset"] = "earliest"

consumer = Consumer(config)

In [8]:
topic = 'ecommerce'
consumer.subscribe([topic])

def process_message(message):
    try:
        if message.error():
            if(message.error.code()) == KafkaError._PARTITION_EOF:
                print('End of partition reached {0}/{1}')
            else:
                raise KafkaException(message.error())
        else:
            key = message.key().decode('utf-8')
            value = json.loads(message.value().decode('utf-8'))

            print(f'Recieved message: Key: {key} , value: {value}')
    except Exception as e:
        print(f'Error consuming/processing message: {e}')

# Poll Message Continously

try:
    print(f'Listening for messages, press ctrl + c in output cell to exit')

    while True:
        message = consumer.poll(timeout=1.0)
        if (message):
            process_message(message)
except KeyboardInterrupt:
    print('Interrupt by user, shutting dwon consumer')
finally:
    consumer.close()

Listening for messages, press ctrl + c in output cell to exit
Recieved message: Key: 123 , value: {'1': 'purinjatha'}
Interrupt by user, shutting dwon consumer
